# Configurations for Classification Tasks

In [1]:
import os,nbloader,warnings,sys,time
warnings.filterwarnings("ignore")
import numpy as np
import tensorflow as tf 
import tensorflow.contrib.slim as slim
from random import shuffle
from define_choiceNet_cls import choiceNet_cls_class
from define_cnn_cls import cnn_cls_class
from multiprocessing import Process,Manager
from util import grid_maker,gpusession,load_mnist_with_noise,get_properIdx
if __name__ == "__main__":
    print ("TensorFlow version is [%s]."%(tf.__version__))

TensorFlow version is [1.4.1].


### Main Configurations

In [3]:
methodList = ['CN','CNN']
useMixupList = [False]
outlierRatioList = [0.5,0.90,0.95]
errTypeList = ['rp','rs','b'] # Random permutation (rp), random shuffle (rs), and biased
def get_mnist_config(_processID=0,_maxProcessID=8,_maxGPU=8,_DO_SHUFFLE=False):
    _G = grid_maker(methodList,useMixupList,outlierRatioList,errTypeList)
    # Get current configurations
    _ids = get_properIdx(_processID,_maxProcessID,_nTask=_G.nIter)
    _paramsList = list(_G.paramList[i] for i in _ids) 
    # Set GPU ID
    _GPU_ID = (_processID % _maxGPU)
    # Suffle if required
    if _DO_SHUFFLE:
        shuffle(_paramsList)
    return _paramsList,_GPU_ID

### Common Configurations

In [5]:
def get_mnist_common_config():
    kmix,rho_ref_train,tau_inv,pi1_bias,logSigmaZval = 10,0.95,1e-4,0.0,-2 # 10,0.95,1e-4,0.0,-2
    logsumexp_coef,kl_reg_coef,l2_reg_coef = 1e-2,1e-4,1e-5 # 0.1,0.1,1e-5
    USE_INPUT_BN,USE_RESNET,USE_GAP,USE_KENDALL_LOSS = False,True,False,False
    maxEpoch = 40
    return kmix,rho_ref_train,tau_inv,pi1_bias,logSigmaZval \
            ,logsumexp_coef,kl_reg_coef,l2_reg_coef \
            ,USE_INPUT_BN,USE_RESNET,USE_GAP,USE_KENDALL_LOSS,maxEpoch

### MNIST Train Wrapper

In [6]:
def train_wrapper_mnist(_paramsList,_GPU_ID,_DO_PRINT=True):
     # Train on different configurations 
    for pIdx,params in enumerate(_paramsList): # For all current configurations
        # Parse current configuration
        method,useMixup,outlierRatio,errType \
            = params[0],params[1],params[2],params[3]
        # Get common parameters
        xdim,ydim,hdims,filterSizes,max_pools,feat_dim = [28,28,1],10,[64,64],[3,3],[2,2],256
        actv,bn,VERBOSE = tf.nn.relu,slim.batch_norm,False
        kmix,rho_ref_train,tau_inv,pi1_bias,logSigmaZval \
            ,logsumexp_coef,kl_reg_coef,l2_reg_coef \
            ,USE_INPUT_BN,USE_RESNET,USE_GAP,USE_KENDALL_LOSS,maxEpoch \
            = get_mnist_common_config()
        # Load CIFAR-10 with outlier
        trainimg,trainlabel,testimg,testlabel,valimg,vallabel \
            = load_mnist_with_noise(_errType=errType,_outlierRatio=outlierRatio,_seed=0)
        # ===== // =====
        if method == 'CN':
            tf.reset_default_graph(); tf.set_random_seed(0)
            CN = choiceNet_cls_class(_name=('mnist_%s_err%.0f_%s_choiceNet'%(errType,outlierRatio*100
                                                ,('mixup' if useMixup else 'basic') ))
                      ,_xdim=xdim,_ydim=ydim,_hdims=hdims,_filterSizes=filterSizes
                      ,_max_pools=max_pools,_feat_dim=feat_dim,_kmix=kmix,_actv=actv,_bn=slim.batch_norm
                      ,_rho_ref_train=rho_ref_train,_tau_inv=tau_inv,_pi1_bias=pi1_bias,_logSigmaZval=logSigmaZval
                      ,_logsumexp_coef=logsumexp_coef,_kl_reg_coef=kl_reg_coef,_l2_reg_coef=l2_reg_coef
                      ,_USE_INPUT_BN=USE_INPUT_BN,_USE_RESNET=USE_RESNET,_USE_GAP=USE_GAP,_USE_KENDALL_LOSS=USE_KENDALL_LOSS
                      ,_USE_MIXUP=useMixup,_GPU_ID=_GPU_ID
                      ,_VERBOSE=VERBOSE)
            sess = gpusession(); sess.run(tf.global_variables_initializer()) 
            CN.train(_sess=sess,_trainimg=trainimg,_trainlabel=trainlabel
                     ,_testimg=testimg,_testlabel=testlabel,_valimg=valimg,_vallabel=vallabel
                     ,_maxEpoch=maxEpoch,_batchSize=256,_lr=1e-5,_kp=0.95
                     ,_LR_SCHEDULE=True,_PRINT_EVERY=100,_SAVE_BEST=True)
            sess.close()
        elif method == 'CNN':
            tf.reset_default_graph(); tf.set_random_seed(0)
            CNN = cnn_cls_class(_name=('mnist_%s_err%.0f_%s_cnn'%(errType,outlierRatio*100
                                                ,('mixup' if useMixup else 'basic')))
                                ,_xdim=xdim,_ydim=ydim,_hdims=hdims,_filterSizes=filterSizes
                                ,_max_pools=max_pools,_feat_dim=feat_dim,_actv=actv,_bn=slim.batch_norm
                                ,_l2_reg_coef=1e-5
                                ,_USE_INPUT_BN=USE_INPUT_BN,_USE_RESNET=USE_RESNET,_USE_GAP=USE_GAP
                                ,_USE_MIXUP=useMixup,_GPU_ID=_GPU_ID
                                ,_VERBOSE=VERBOSE)
            sess = gpusession(); sess.run(tf.global_variables_initializer()) 
            CNN.train(_sess=sess,_trainimg=trainimg,_trainlabel=trainlabel
                      ,_testimg=testimg,_testlabel=testlabel,_valimg=valimg,_vallabel=vallabel
                      ,_maxEpoch=maxEpoch,_batchSize=256,_lr=1e-5,_kp=0.95
                      ,_LR_SCHEDULE=True,_PRINT_EVERY=100,_SAVE_BEST=True)
            sess.close() 
        else:
            print ('Unknown method: [%s]'%(method))

### Worker Class

In [7]:
class worker_class(Process):
    def __init__(self,_idx=0,_maxProcessID=8,_maxGPU=8,_name='worker',_FLAG='',_period=1,_maxTick=5,_VERBOSE=True):
        Process.__init__(self)
        # Initialize class
        self.idx = _idx
        self.maxProcessID = _maxProcessID
        self.maxGPU = _maxGPU
        self.name = _name
        self.FLAG = _FLAG # Running flag 
        self.VERBOSE = _VERBOSE
        # Initialize Process 
        self.setName = 'T_'+self.name 
        self.args = (self.FLAG,)
        self.daemon = True # Make sure that each child is killed when the parent is dead. (?!)
        # Print
        if self.VERBOSE:
            print ("[%s] Instantiated."%(self.name))
    def run(self):
        print ("Starting [%s]"%(self.name))
        time.sleep(1e-1)
        # Get configurations
        paramsList,GPU_ID = get_mnist_config(self.idx,self.maxProcessID,self.maxGPU)
        print ("processID:[%d/%d] GPU_ID:[%d] #Config:[%d]"
               %(self.idx,self.maxProcessID,GPU_ID,len(paramsList)))
        # Run
        train_wrapper_mnist(_paramsList=paramsList,_GPU_ID=GPU_ID,_DO_PRINT=False)
        print ("[%s] Done."%(self.name))
if __name__ == "__main__":
    print ("worker_class defined.")

worker_class defined.
